# Replacing TCJA CTC expansion with child allowance

In [1]:
from taxcalc import *

Load Ernie Tedeschi's weights file.

In [2]:
tedeschi_weights = pd.read_csv('cps_weights_2026.csv')

In [44]:
# Initiate baseline calculator.
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
calc.records.s006 = tedeschi_weights.s006
calc.advance_to_year(2018)
calc.calc_all()
# Initiate calculator to which reforms are applied.
recs_house = Records.cps_constructor()
calc_house = Calculator(records=recs_house, policy=Policy())
calc_house.records.s006 = tedeschi_weights.s006
# Initiate calculator to which reforms are applied.
recs_house_no_ctc_exp = Records.cps_constructor()
calc_house_no_ctc_exp = Calculator(records=recs_house_no_ctc_exp, policy=Policy())
calc_house_no_ctc_exp.records.s006 = tedeschi_weights.s006
# Initiate UBI calculator (ca = child allowance)
recs_house_ca = Records.cps_constructor()
calc_house_ca = Calculator(records=recs_house_ca, policy=Policy())
calc_house_ca.records.s006 = tedeschi_weights.s006

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [46]:
#ubi_reform = calc_x.read_json_param_objects('Child_UBI.json', 'SampleBehavior.json')
house_reform = calc_house.read_json_param_objects('TCJA_House.json', 'SampleBehavior.json')
house_reform_no_ctc_exp = calc_house_no_ctc_exp.read_json_param_objects(
    'TCJA_House_no_CTC_expansion.json', 'SampleBehavior.json')
house_reform_ca = calc_house_ca.read_json_param_objects(
    'TCJA_House_child_allowance.json', 'SampleBehavior.json')
#senate_reform = calc_x.read_json_param_objects('TCJA_Senate.json', 'SampleBehavior.json')
#house_reform_ubi = calc_x.read_json_param_objects('TCJA_House_Child_UBI.json', 'SampleBehavior.json')
#senate_reform_ubi = calc_x.read_json_param_objects('TCJA_House_Child_UBI.json', 'SampleBehavior.json')

In [47]:
calc_house.policy.implement_reform(house_reform['policy'])
calc_house.consumption.update_consumption(house_reform['consumption'])
calc_house.advance_to_year(2018)

In [48]:
calc_house_no_ctc_exp.policy.implement_reform(house_reform_no_ctc_exp['policy'])
calc_house_no_ctc_exp.consumption.update_consumption(house_reform_no_ctc_exp['consumption'])
calc_house_no_ctc_exp.advance_to_year(2018)

Baseline = No CTC expansion

Reform = Normal House bill

In [13]:
calc_house.behavior.update_behavior(house_reform['behavior'])
calc_response = Behavior.response(calc_house_no_ctc_exp, calc_house)

Viewing the results

In [14]:
from taxcalc.utils import *

In [15]:
# Combined is combined tax liability while s006 is weight.
baseline = (calc_house_no_ctc_exp.records.combined * calc_house_no_ctc_exp.records.s006).sum()
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

Combined Liability - Baseline: 2511627853030.14
Combined Liability - Reform:   2459480619903.98
-----------------------------------------------
Difference:                     -52147233126.16


In [17]:
create_difference_table(calc_house_no_ctc_exp.records, calc_house.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58","29,337.88",0.17,0.00,0.00,-0.28,"-4,799,572.81",0.01,0.00,-0.00
1,"16,988,285.70","641,263.00",3.77,0.00,0.00,-3.39,"-57,531,392.43",0.12,-0.02,0.02
2,"16,990,004.05","1,983,739.85",11.68,0.00,0.00,-14.13,"-240,022,134.75",0.50,-0.07,0.07
3,"16,989,656.05","3,232,598.28",19.03,0.00,0.00,-34.95,"-593,847,775.86",1.24,-0.13,0.13
4,"16,989,954.66","3,647,576.68",21.47,0.00,0.00,-68.20,"-1,158,795,338.23",2.42,-0.21,0.21
5,"16,989,223.86","3,871,601.01",22.79,0.00,0.00,-111.74,"-1,898,362,398.00",3.96,-0.28,0.28
6,"16,989,703.32","4,404,615.82",25.93,0.00,0.00,-202.91,"-3,447,403,648.40",7.19,-0.39,0.39
7,"16,989,484.52","8,621,245.07",50.74,0.00,0.00,-403.34,"-6,852,531,979.67",14.30,-0.59,0.59
8,"16,989,613.44","12,587,085.56",74.09,0.00,0.00,-907.69,"-15,421,233,256.57",32.18,-0.92,0.92
9,"16,989,497.48","9,872,611.52",58.11,0.00,0.00,"-1,074.22","-18,250,392,535.35",38.08,-0.76,0.76


In [20]:
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [24]:
mtr_plot_data = mtr_graph_data(calc_house_no_ctc_exp, calc_response)

In [25]:
show(xtr_graph_plot(mtr_plot_data))

In [26]:
atr_plot_data = atr_graph_data(calc_house_no_ctc_exp, calc_response)

In [27]:
show(xtr_graph_plot(atr_plot_data))

In [28]:
multiyear_diagnostic_table(calc_response, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,518.1","11,002.5","11,490.2","12,015.9","12,578.5","13,165.0","13,778.5","14,421.4","15,091.4"
Itemizers (#m),13.2,14.2,15.2,16.2,17.2,18.2,19.3,20.3,21.5
Itemized Deduction ($b),309.1,345.1,379.8,417.7,458.1,501.5,548.5,598.0,653.7
Standard Deduction Filers (#m),138.8,139.8,141.0,142.1,143.3,144.4,145.5,146.7,147.7
Standard Deduction ($b),"2,382.4","2,449.7","2,523.9","2,601.9","2,683.8","2,766.1","2,851.2","2,938.4","3,026.4"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"8,177.6","8,568.3","8,957.1","9,377.6","9,829.3","10,301.5","10,795.1","11,313.6","11,852.4"
Regular Tax ($b),"1,408.4","1,485.2","1,560.1","1,641.2","1,727.0","1,815.4","1,907.2","2,003.7","2,102.5"
AMT Income ($b),"10,258.3","10,712.1","11,170.4","11,664.1","12,192.6","12,742.3","13,316.0","13,916.9","14,539.7"


## Distribute as child allowance

Find child allowance amount.

In [29]:
diff

-52147233126.160645

`nu18` [indicates](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/records_variables.json#L512) includes people under 18 ([does it include 18yos?](https://github.com/open-source-economics/Tax-Calculator/issues/1669)), and is what the UBI reform refers to.

This differs from CTC-eligible children, which [also includes](https://www.irs.gov/credits-deductions/individuals/earned-income-tax-credit/qualifying-child-rules) these two groups:

> At the end of the filing year, your child was younger than you (or your spouse if you file a joint return) younger than 24 and a full-time student
> At the end of the filing year, your child was any age and permanently and totally disabled

This could be one reason households would be better off with CTC expansion.

In [42]:
nu18 = sum(calc_response.records.nu18 * calc_response.records.s006)

In [43]:
nu18
diff / nu18

-692.01780406667297

In [ ]:
calc_response.records.combined * calc_response.records.s006

### Analyze

Baseline: House TCJA.
Reform: House TCJA replacing CTC expansion with child allowance.

In [49]:
calc_house_ca.policy.implement_reform(house_reform_ca['policy'])
calc_house_ca.consumption.update_consumption(house_reform_ca['consumption'])
calc_house_ca.advance_to_year(2018)

In [50]:
calc_house_ca.behavior.update_behavior(house_reform_ca['behavior'])
calc_response_ca = Behavior.response(calc_house, calc_house_ca)

In [51]:
# Combined is combined tax liability while s006 is weight.
baseline = (calc_house.records.combined * calc_house.records.s006).sum()
reformed = (calc_response_ca.records.combined * calc_response_ca.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

Combined Liability - Baseline: 2463702932998.07
Combined Liability - Reform:   2526001069704.94
-----------------------------------------------
Difference:                      62298136706.88
